In [1]:
import pandas as pd
import datetime

def process_csv(filepath, product_index):
    """
    This function takes the file path and product index as inputs and returns the filtered trade data dataframe
    for a particular product.
    
    Parameters:
    filepath (str) : file path for the trade data file
    product_index (int) : index of the product in the list of all products
    
    Returns:
    df_product_filtered (pandas dataframe) : dataframe for the filtered product
    """
    
    # Read the CSV file and convert the "TIME_PERIOD" column to datetime
    trade_data = pd.read_csv(filepath)
    trade_data["TIME_PERIOD"] = pd.to_datetime(trade_data["TIME_PERIOD"], format = '%Y-%m').dt.date
    
    # Rename columns and remove duplicates
    trade_data = trade_data.rename(columns={"partner": "PARTNER_codes", "declarant": "DECLARANT_codes"})
    trade_data = trade_data.drop_duplicates()
    
    # Get the list of all products
    products = trade_data["product"].drop_duplicates().to_list()
    
    # Filter the data for a particular product
    product_filter = products[product_index]
    
    # Read the partner countries data
    partner_countries = pd.read_csv("trade_data/partners.csv", sep = ";")
    
    # Merge the trade data and partner country data on the "PARTNER_codes" column
    trade_data = pd.merge(trade_data, partner_countries, on = ['PARTNER_codes'], how = 'inner')
    
    # Pivot the trade data to create a multi-index dataframe
    trade_data_pivot = trade_data.pivot(index=['PARTNER_Labels', 'TIME_PERIOD', 'product'], 
                                  columns='indicators', 
                                  values=['OBS_VALUE'])
    
    # Flatten the column names
    trade_data_pivot.columns = trade_data_pivot.columns.map('_'.join)
    
    # Reset the index and remove missing values
    trade_data_pivot = trade_data_pivot.reset_index()
    trade_data_pivot = trade_data_pivot.dropna()
    
    # Filter the data for the selected product
    df = trade_data_pivot
    df_product_filtered = df[df['product'] == product_filter]

    
    return df_product_filtered

#Declare DataFrames for all good/productcode combinations
df_sunflower_oil_0 = process_csv("trade_data/sunflower_oil.csv", 0)
df_sunflower_oil_0['price'] = df_sunflower_oil_0['OBS_VALUE_VALUE_1000EURO'] / df_sunflower_oil_0['OBS_VALUE_QUANTITY_TON']

df_sunflower_oil_1 = process_csv("trade_data/sunflower_oil.csv", 1)
df_sunflower_oil_1['price'] = df_sunflower_oil_1['OBS_VALUE_VALUE_1000EURO'] / df_sunflower_oil_1['OBS_VALUE_QUANTITY_TON']



In [2]:
df_sunflower = df_sunflower_oil_0.merge(df_sunflower_oil_1, on=["PARTNER_Labels", "TIME_PERIOD"], how='outer')
df_sunflower = df_sunflower.fillna(0)

df_sunflower

,PARTNER_Labels,TIME_PERIOD,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,price_y
0,Argentina,2005-01-01,15121191.0,21086.3,10489.97,0.497478,0.0,0.0,0.00,0.000000
1,Argentina,2005-02-01,15121191.0,18325.9,8946.33,0.488180,0.0,0.0,0.00,0.000000
2,Argentina,2005-03-01,15121191.0,25661.5,12741.53,0.496523,0.0,0.0,0.00,0.000000
3,Argentina,2005-04-01,15121191.0,33839.2,16665.38,0.492487,0.0,0.0,0.00,0.000000
4,Argentina,2005-05-01,15121191.0,41051.1,19921.35,0.485282,0.0,0.0,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...
4479,United States,2020-05-01,0.0,0.0,0.00,0.000000,15121990.0,0.1,0.06,0.600000
4480,United States,2020-07-01,0.0,0.0,0.00,0.000000,15121990.0,3.9,4.02,1.030769
4481,United States,2021-01-01,0.0,0.0,0.00,0.000000,15121990.0,0.1,0.07,0.700000
4482,United States,2021-02-01,0.0,0.0,0.00,0.000000,15121990.0,1.9,3.15,1.657895


## AGGREGATING BOTH PRODUCT CATEGORIES

In [3]:
df_sunflower['OBS_VALUE_QUANTITY_TON'] = df_sunflower['OBS_VALUE_QUANTITY_TON_x'] + df_sunflower['OBS_VALUE_QUANTITY_TON_y']
df_sunflower['OBS_VALUE_VALUE_1000EURO'] = df_sunflower['OBS_VALUE_VALUE_1000EURO_x'] + df_sunflower['OBS_VALUE_VALUE_1000EURO_y']
df_sunflower['price'] = df_sunflower['OBS_VALUE_VALUE_1000EURO'] / df_sunflower['OBS_VALUE_QUANTITY_TON']

df_sunflower

,PARTNER_Labels,TIME_PERIOD,product_x,OBS_VALUE_QUANTITY_TON_x,OBS_VALUE_VALUE_1000EURO_x,price_x,product_y,OBS_VALUE_QUANTITY_TON_y,OBS_VALUE_VALUE_1000EURO_y,price_y,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price
0,Argentina,2005-01-01,15121191.0,21086.3,10489.97,0.497478,0.0,0.0,0.00,0.000000,21086.3,10489.97,0.497478
1,Argentina,2005-02-01,15121191.0,18325.9,8946.33,0.488180,0.0,0.0,0.00,0.000000,18325.9,8946.33,0.488180
2,Argentina,2005-03-01,15121191.0,25661.5,12741.53,0.496523,0.0,0.0,0.00,0.000000,25661.5,12741.53,0.496523
3,Argentina,2005-04-01,15121191.0,33839.2,16665.38,0.492487,0.0,0.0,0.00,0.000000,33839.2,16665.38,0.492487
4,Argentina,2005-05-01,15121191.0,41051.1,19921.35,0.485282,0.0,0.0,0.00,0.000000,41051.1,19921.35,0.485282
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4479,United States,2020-05-01,0.0,0.0,0.00,0.000000,15121990.0,0.1,0.06,0.600000,0.1,0.06,0.600000
4480,United States,2020-07-01,0.0,0.0,0.00,0.000000,15121990.0,3.9,4.02,1.030769,3.9,4.02,1.030769
4481,United States,2021-01-01,0.0,0.0,0.00,0.000000,15121990.0,0.1,0.07,0.700000,0.1,0.07,0.700000
4482,United States,2021-02-01,0.0,0.0,0.00,0.000000,15121990.0,1.9,3.15,1.657895,1.9,3.15,1.657895


In [4]:
# group by TIME_PERIOD and aggregate the OBS_VALUE_QUANTITY_TON and OBS_VALUE_VALUE_1000EURO columns by summing
grouped_df = df_sunflower.groupby('TIME_PERIOD').agg({'OBS_VALUE_QUANTITY_TON': 'sum', 'OBS_VALUE_VALUE_1000EURO': 'sum'})

# calculate globalprice
globalprice = pd.DataFrame(grouped_df['OBS_VALUE_VALUE_1000EURO'] / grouped_df['OBS_VALUE_QUANTITY_TON'])
# rename the column to 'globalprice'
globalprice = globalprice.rename(columns={0: 'globalprice'})


globalprice = globalprice.reset_index()
globalprice = globalprice.rename(columns={'index': 'TIME_PERIOD'})


globalprice

,TIME_PERIOD,globalprice
0,2005-01-01,0.547673
1,2005-02-01,0.544328
2,2005-03-01,0.561141
3,2005-04-01,0.530829
4,2005-05-01,0.511798
...,...,...
209,2022-06-01,1.535339
210,2022-07-01,1.841942
211,2022-08-01,1.759469
212,2022-09-01,1.613330


## CLEANING EXTREMES

In [ ]:
import numpy as np

# Delete all inf and -inf
df_sunflower = df_sunflower.replace([np.inf, -np.inf], np.nan)\

# Define a function to replace extreme high values with NaN
def replace_outliers(df, column):
    z = np.abs((df[column] - df[column].mean()) / df[column].std()) # Calculate z-score for the column
    outliers = df[z > 3][column] # Identify extreme high values with a z-score greater than 3
    median = df[column].median() # Calculate the median of the column
    df.loc[z > 3, column] = np.nan # Replace extreme high values with NaN

# Define a list of columns to perform outlier treatment on
columns = ['price_x','price_y', 'price']

# Iterate over the columns and call the replace_outliers function for each one
for column in columns:
    replace_outliers(df_sunflower, column)
    

In [ ]:
df_sunflower

## WEATHER DATA

In [ ]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the weather file
df_global_temperature = pd.read_csv('weather_data/global_temperature_data.csv')

# Rename the 'YearMonth' column to 'TIME_PERIOD'
df_global_temperature.rename(columns={'yearmonth': 'TIME_PERIOD', 'partner_labels': 'PARTNER_Labels'}, inplace=True)

# Convert the TIME_PERIOD column to datetime format
df_global_temperature['TIME_PERIOD'] = pd.to_datetime(df_global_temperature['TIME_PERIOD'], format='%Y%m')

# Convert the TIME_PERIOD column to the desired format
df_global_temperature['TIME_PERIOD'] = df_global_temperature['TIME_PERIOD'].dt.strftime('%Y-%m-%d')

# Drop columns
df_global_temperature.drop(columns=['partner_code', 'weather_countrycode', 'weather_countryname', 'iso_countrycode'], inplace=True)

df_global_temperature['TIME_PERIOD'] = pd.to_datetime(df_global_temperature['TIME_PERIOD'])
df_global_temperature

## OIL DATA

In [ ]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the 'oil_price_index_2015_oecd.xlsx' file
df_oil_index = pd.read_excel('oil_prices/oil_price_index_2015_oecd.xlsx', skiprows=[0,1,2,3,4, 9012, 9013, 9014, 9015])
# Forward fill the 'Country' column
df_oil_index['Country'].fillna(method='ffill', inplace=True)
# Rename columns
df_oil_index.rename(columns={'Unnamed: 3': 'CPI_ENERGY'}, inplace=True)
df_oil_index.rename(columns={'Unnamed: 4': 'CPI_ENERGY_HARMONISED'}, inplace=True)
# Delete the 'Unnamed: 2' column
df_oil_index.drop(columns=['Unnamed: 2'], inplace=True)
# Convert the 'Time' column to datetime format
df_oil_index['Time'] = pd.to_datetime(df_oil_index['Time'], format='%b-%Y')
df_oil_index.rename(columns={'Time': 'TIME_INDEX'}, inplace=True)

#Clean import CMO-Historical-Data-Monthly.xlsx

df_commodities_price = pd.read_excel('oil_prices/CMO-Historical-Data-Monthly.xlsx', sheet_name = 'Monthly Prices', skiprows=[0,1,2,3,4,5])
# Rename columns
df_commodities_price.rename(columns={'Unnamed: 0': 'TIME_PERIOD'}, inplace=True)
df_commodities_price.rename(columns={'CRUDE_PETRO': 'OIL_PRICE $/bbl'}, inplace=True)
df_commodities_price['TIME_PERIOD'] = pd.to_datetime(df_commodities_price['TIME_PERIOD'], format='%YM%m')

df_oil_price_full = pd.DataFrame(df_commodities_price, columns=['TIME_PERIOD', 'OIL_PRICE $/bbl'])

df_oil_price = df_oil_price_full.loc[(df_oil_price_full['TIME_PERIOD'] >= '2005-01-01') & (df_oil_price_full['TIME_PERIOD'] <= '2022-10-01')]
df_oil_price

## FUTURES PRICES (SUNFLOWER OIL)

In [ ]:
# Extracted from https://www.investing.com/commodities/high-oil-content-sunflower-seed-historical-data
import pandas as pd
import datetime

#Clean import High Oil Content Sunflower Seed Futures Historical Data.csv

# Read the data from the file
df_futures_sunflower_seed = pd.read_csv('futures_data/High Oil Content Sunflower Seed Futures Historical Data.csv')

# Rename the 'date' column to 'TIME_PERIOD'
df_futures_sunflower_seed.rename(columns={'Date': 'TIME_PERIOD', 'Price': 'Sunflower Seed Price Futures'}, inplace=True)

df_futures_sunflower_seed['TIME_PERIOD'] = pd.to_datetime(df_futures_sunflower_seed['TIME_PERIOD'], format='%d/%m/%Y')
df_futures_sunflower_seed['TIME_PERIOD'] = df_futures_sunflower_seed['TIME_PERIOD'].dt.strftime('%Y/%m/%d')

df_futures_sunflower_seed.drop(columns=['Open','High','Low','Vol.','Change %'], inplace=True)

df_futures_sunflower_seed.sort_values(by='TIME_PERIOD', ascending=True, inplace=True)

# Replace commas with dots in 'Sunflower Seed Price Futures' column
df_futures_sunflower_seed['Sunflower Seed Price Futures'] = df_futures_sunflower_seed['Sunflower Seed Price Futures'].str.replace(',', '.')

# Convert 'Sunflower Seed Price Futures' column to float datatype
df_futures_sunflower_seed['Sunflower Seed Price Futures'] = df_futures_sunflower_seed['Sunflower Seed Price Futures'].astype(float)


df_futures_sunflower_seed

## MACROECONOMIC DATA

In [ ]:
#import datetime
import numpy
import openpyxl
#import data
path_file_1_p = "macroeconomic_data/Population_projections.xlsx"
population_data = pd.read_excel(path_file_1_p)

path_file_1 = "macroeconomic_data/macro_economic_indicators.csv"
economic_indicators = pd.read_csv(path_file_1, sep = ";", decimal=".")

#rename column for merge later
population_data = population_data.rename(columns = {'Country Name': 'PARTNER_Labels'})
economic_indicators = economic_indicators.rename(columns = {'country_name': 'PARTNER_Labels' ,'Time':'TIME_PERIOD' })
#this is a quick fix
macro_economic_indicators = economic_indicators
# Convert year to datetime with year and month
macro_economic_indicators['year'] = pd.to_datetime(macro_economic_indicators['year'], format='%Y').dt.to_period('M')
# Set the date as the index
macro_economic_indicators.set_index('year', inplace=True)

#make df for monthly data
monthly_data = macro_economic_indicators.iloc[0:0].copy()


number_of_col = macro_economic_indicators.shape[1]
nan_row = numpy.empty((number_of_col,))
nan_row[:] = numpy.nan

#add na rows for 11 months in a year
n_row = macro_economic_indicators.shape[0] 
for i in range(n_row):
    print(i)
    holder_for_rows = macro_economic_indicators.iloc[i,:]
    current_country = macro_economic_indicators.iloc[i,0] 
    current_country_code = macro_economic_indicators.iloc[i,1] 
    for j in range(12):
        index_3 = 12 * i + j
        if j == 0 :
            monthly_data.loc[index_3] = holder_for_rows   
        else:
            monthly_data.loc[index_3] = nan_row
            monthly_data.iloc[index_3,0] = current_country
            monthly_data.iloc[index_3,1] = current_country_code

#slects als the column with numbers
c = monthly_data.columns[3:12]
monthly_data[c] = monthly_data[c].apply(pd.to_numeric)

n_countries = len(population_data['Country Code'].unique())
#fills in the date column
for i in range(n_countries-1):
    index_1 = 0   + 204*i
    index_2 = 204 + 204*i
    monthly_data.iloc[index_1:index_2,:] = monthly_data.iloc[index_1:index_2,:].interpolate().copy()
    #adding dates 
    monthly_data.iloc[index_1:index_2,2] = pd.period_range("2005-01", "2021-12", freq='M')

In [ ]:
import pandas as pd
import datetime
import numpy as np
import openpyxl
#import data
path_file_1_p = "macroeconomic_data/Population_projections.xlsx"
population_data = pd.read_excel(path_file_1_p)

#delete the last 5 empty rows 
num_rows = len(population_data)
population_data = population_data.drop(population_data.index[num_rows-5:num_rows])
#rename a column 
population_data = population_data.rename(columns = {'Country Name': 'PARTNER_Labels','Time':'TIME_PERIOD'})
population_data = population_data.rename(columns = {'Population, total [SP.POP.TOTL]': 'A'})

#get's the index s of the columns with string
df = population_data
string_indices = df['A'][df['A'].apply(lambda x: isinstance(x, str))].index


#gets th countries corresponging to the index s 
lll = len(string_indices)
LIST_of_del_countries = [None] * lll

counter = 0 
for i in range(lll):
    LIST_of_del_countries[counter] = population_data.iloc[(string_indices[i]),0]
    counter = counter +1

list1 = list(set(LIST_of_del_countries))

print(df.shape)
print(list1)
for j in range(4000):
  
    if df.iloc[j,0] in list1:
        df = df.drop(index=[j])
        print(df.shape)

df = df.replace('..', 0)
print(df)
# boolean indexing to select rows to delete
idx = df['PARTNER_Labels'].str.contains('IDA blend')

# drop rows using boolean indexing
df = df.drop(df[idx].index)

idx = df['PARTNER_Labels'].str.contains('IDA only')
df = df.drop(df[idx].index)

idx = df['PARTNER_Labels'].str.contains('IDA total')
df = df.drop(df[idx].index)

for name in list1:
    idx = df['PARTNER_Labels'].str.contains(name)
    df = df.drop(df[idx].index)


testing= df.copy()
excel_test = pd.ExcelWriter("population_data.xlsx")
testing.to_excel(excel_test)
excel_test.save()
#make the 
monthly_data_p = population_data.iloc[0:0].copy()

number_of_col = df.shape[1]
nan_row = np.empty((number_of_col,))
nan_row[:] = np.nan

n_row = df.shape[0] 
for i in range(n_row):
    print(i)
    holder_for_rows = df.iloc[i,:]
    current_country = df.iloc[i,0]
    current_country_code = df.iloc[i,1] 
    for j in range(12):
        index_3 = 12 * i + j
        if j == 0 :
            monthly_data_p.loc[index_3] = holder_for_rows   
        else:
            monthly_data_p.loc[index_3] = nan_row
            monthly_data_p.iloc[index_3,0] = current_country
            monthly_data_p.iloc[index_3,1] = current_country_code 


# testing= monthly_data_p.copy()
# excel_test = pd.ExcelWriter("dopper2.xlsx")
# testing.to_excel(excel_test)
# excel_test.save()

c_p = population_data.columns[4]
monthly_data_p[c_p] = monthly_data_p[c_p].apply(pd.to_numeric)
n_countries = len(df['Country Code'].unique())
#fills in the date column
for i in range(n_countries-10):
    index_1 = 0   + 228*i
    index_2 = 228 + 228*i
    monthly_data_p.iloc[index_1:index_2,4] = monthly_data_p.iloc[index_1:index_2,4].interpolate().copy()
    #adding dates 
    monthly_data_p.iloc[index_1:index_2,2] = pd.period_range("2005-01", "2023-12", freq='M')

# testing= monthly_data_p.copy()
# excel_test = pd.ExcelWriter("dopper3.xlsx")
# testing.to_excel(excel_test)
# excel_test.save()


In [ ]:
monthly_data_p.head(214)

In [ ]:
monthly_data_p  = monthly_data_p.rename(columns = {'Country Code': 'country_code'})
# convert Period and float values to string format, and then to datetime
monthly_data_p ['TIME_PERIOD'] = monthly_data_p ['TIME_PERIOD'].apply(lambda x: str(x) if isinstance(x, pd.Period) else x)
monthly_data_p ['TIME_PERIOD'] = pd.to_datetime(monthly_data_p ['TIME_PERIOD'], errors='coerce')
monthly_data_p.head(214)

In [ ]:
monthly_data  = monthly_data.rename(columns = {'timecode': 'TIME_PERIOD'})
# convert Period and float values to string format, and then to datetime
monthly_data['TIME_PERIOD'] = monthly_data['TIME_PERIOD'].apply(lambda x: str(x) if isinstance(x, pd.Period) else x)
monthly_data['TIME_PERIOD'] = pd.to_datetime(monthly_data['TIME_PERIOD'], errors='coerce')

monthly_data.head(206)

In [ ]:
merged_monthly_data_df = monthly_data.merge(monthly_data_p, on=['country_code', 'TIME_PERIOD'], how='right')
merged_monthly_data_df = merged_monthly_data_df.drop('PARTNER_Labels_y', axis=1)
merged_monthly_data_df = merged_monthly_data_df.rename(columns={'PARTNER_Labels_x': 'PARTNER_Labels', 'A': 'population_projections'})
merged_monthly_data_df

In [ ]:
df_macroeconomic = merged_monthly_data_df
df_macroeconomic.head(214)

## MERGING DATA

### Merged trade data, oil price

In [ ]:
df_sunflower['TIME_PERIOD'] = pd.to_datetime(df_sunflower['TIME_PERIOD'])
df_merged_trade_oil = df_oil_price.merge(df_sunflower, on='TIME_PERIOD', how='outer')
df_merged_trade_oil.head(30)

### Merged trade data, oil price, weather data

In [ ]:
df_merged_trade_oil_weather = df_merged_trade_oil.merge(df_global_temperature, on=['TIME_PERIOD', 'PARTNER_Labels'], how='outer')
df_merged_trade_oil_weather

### Merged trade data, oil price, weather, future prices data

In [ ]:
df_futures_sunflower_seed['TIME_PERIOD'] = pd.to_datetime(df_futures_sunflower_seed['TIME_PERIOD'])
df_merged_trade_oil_weather_futures = df_merged_trade_oil_weather.merge(df_futures_sunflower_seed, on='TIME_PERIOD', how='outer')
df_merged_trade_oil_weather_futures.head(50)

### Merged trade, oil price, weather, future prices, macroeconomic

In [ ]:
df_merged = df_merged_trade_oil_weather_futures.merge(df_macroeconomic, on=['TIME_PERIOD', 'PARTNER_Labels'], how='left')
df_merged

In [ ]:
df_merged.columns

## CLEANING EXTREME VALUES

In [ ]:
import numpy as np

# Delete all inf and -inf
df_merged = df_merged.replace([np.inf, -np.inf], np.nan)

df_merged.columns

In [ ]:
df_merged['Sunflower Seed Price Futures']

In [ ]:
# Define a function to replace extreme high values with NaN
def replace_outliers(df, column):
    z = np.abs((df[column] - df[column].mean()) / df[column].std()) # Calculate z-score for the column
    outliers = df[z > 3][column] # Identify extreme high values with a z-score greater than 3
    median = df[column].median() # Calculate the median of the column
    df.loc[z > 3, column] = np.nan # Replace extreme high values with NaN

# Define a list of columns to perform outlier treatment on
columns = ['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'OIL_PRICE $/bbl', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'Sunflower Seed Price Futures', 'gdp_current_us', 'population_total', 'unemployment_total', 'renewable_energy_consumption_perc_of_total', 'energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total', 'population_projections']
# Iterate over the columns and call the replace_outliers function for each one
for column in columns:
    replace_outliers(df_merged, column)


df_merged

In [ ]:
# Define a function to replace extreme high values with NaN
def replace_outliers(df, column):
    z = np.abs((df[column] - df[column].mean()) / df[column].std()) # Calculate z-score for the column
    outliers = df[z > 3][column] # Identify extreme high values with a z-score greater than 3
    df.loc[z > 3, column] = np.nan # Replace extreme high values with NaN

# Define a list of columns to perform outlier treatment on
columns = ['price']

# Iterate over the columns and call the replace_outliers function for each one
for column in columns:
    replace_outliers(df_merged, column)



## FULL DATAFRAME (without filtering top countries)

In [ ]:
###FOR NOW
df_sunflower_total = df_merged.pivot_table(index='TIME_PERIOD', columns=['PARTNER_Labels'], values=['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'OIL_PRICE $/bbl', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'Corn Price Futures', 'gdp_current_us', 'population_total','unemployment_total', 'agricultural_land', 'renewable_energy_consumption_perc_of_total','energy_use_kg_of_oil_equivalent_per_capita', 'fossil_fuel_energy_consumption_perc_of_total'], margins=True)
df_sunflower_total

In [ ]:
df_sunflower_total.to_excel("sunflower_total.xlsx")

## ANALYSIS SUNFLOWER OIL IMPORT TO NL (HISTORICAL) & FILTER TOP PARTNERS

In [ ]:
df_sunflower['TIME_PERIOD'] = pd.to_datetime(df_sunflower['TIME_PERIOD'])
mask = (df_sunflower['TIME_PERIOD'] >= '2005-01-01') & (df_sunflower['TIME_PERIOD'] <= '2022-10-01')
df_filtered = df_sunflower.loc[mask]

grouped = df_filtered.groupby('PARTNER_Labels')['OBS_VALUE_QUANTITY_TON'].sum().reset_index()
grouped = grouped.sort_values(by='OBS_VALUE_QUANTITY_TON', ascending=False)
grouped = grouped.reset_index()
grouped['cumulative_sum'] = grouped['OBS_VALUE_QUANTITY_TON'].cumsum()
grouped['cumulative_sum_pct'] = grouped['cumulative_sum'] / grouped['OBS_VALUE_QUANTITY_TON'].sum()

grouped.head(10)        
        


## HANDLE MISSING DATA

In [ ]:
# Create a new DataFrame that only contains the 'price' values of Belgium, Germany and France (complete and in top-6):
df_price = df_sunflower_total[['price']].loc[:, ('price', ['Belgium', 'France', 'Germany'])]
df_price

In [ ]:
# Remove any rows that contain missing values (NaN) in the 'price' column
df_price = df_price.dropna()

In [ ]:
df_price_mean = df_price.mean(axis=1)
df_price_mean

In [ ]:
# Calculate the year-over-year percent change in 'price' for France, Germany and Belgium:
df_price_pct_change = df_price.pct_change()

# Replace NaN values with 0 (there are NaN values for the first year)
df_price_pct_change = df_price_pct_change.fillna(0)
df_price_pct_change

# Calculate the mean percent change in 'price' for each year across France and Germany and Belgium:
df_price_pct_change_mean = df_price_pct_change.mean(axis=1)
df_price_pct_change_mean


In [ ]:
# Create a new DataFrame that only contains the missing 'price' values for other countries:
df_missing_price = df_sunflower_total[['price']].loc[:, ('price', df_sunflower_total['price'].isna().any())]
df_missing_price

In [ ]:
# Fill in the missing 'price' values for each year in the new DataFrame using the mean percent change from France and Germany
df_missing_price = df_missing_price.fillna(method='ffill')
df_missing_price = df_missing_price.fillna(method='bfill')
df_missing_price = df_missing_price.fillna(df_price.iloc[-1])
df_missing_price = df_missing_price.apply(lambda x: x * (1 + df_price_pct_change_mean))

df_missing_price